In [ ]:
from tqdm import tqdm_notebook as tqdm

from kp_utils import *

### Загружаем список фильмов, информацию по которым нужно скачать

In [ ]:
filename_movies_extented = 'kp_movies_extented.csv'

filename_movies = 'kp_movies.csv'
movies = pd.read_csv(filename_movies, delimiter=';', header=0)

In [ ]:
movies

In [ ]:
movies_ids_hrefs = { (int(m[0]), m[1])  for m in movies.values}
len(movies_ids_hrefs)

In [ ]:
# establishing session
s = requests.Session() 
s.headers.update({
        'Referer': 'http://www.kinopoisk.ru',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36',
        'Host': "www.kinopoisk.ru",
        'Upgrade-Insecure-Requests': '1',
        'Cookie': "mobile=no; noflash=true; hideBlocks=0; my_perpages=%7B%2277%22%3A200%2C%2279votes%22%3A200%7D",
        'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7,de;q=0.6',
        'Referer' : 'https://www.kinopoisk.ru',
        'Accept'  : "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        'Connection': 'keep-alive'    
    })

In [ ]:
filename_movies_extented

### Вычтем фильмы, расширенная информация по которым уже скачана

In [ ]:
movies_ids_hrefs = movies_ids_hrefs - get_already_processed_movies(filename_movies_extented)
len(movies_ids_hrefs)

### Процесс краулинга. Долго, конечно

In [ ]:
counter

In [ ]:
import time

movies_ids_hrefs = get_movies_ids_hrefs_to_process(filename_movies) - get_already_processed_movies(filename_movies_extented)
while True:
    movies_batch = list()
    previous_film_url = base_url
    for i, m in enumerate(tqdm(movies_ids_hrefs)):
        s.headers.update({
            'Referer' : previous_film_url,
        })
        movie_id   = m[0]
        movie_href = m[1]

        try:
            html_movie = get_movie_html(movie_href, s)
        except Exception as e:
            html_movie = None
            counter['get_movie_html: %s'%str(e)] += 1

        if html_movie is not None:
            movie       = parse_movie_html(html_movie)
            movie['id']   = movie_id
            movie['href'] = movie_href
            movies_batch.append(movie)
        else:
            counter['html_movie is None'] += 1
            continue

        if i > 0 and i % 1000 == 0:
            write_batch_movies(movies_batch, filename_movies_extented)
            movies_batch = list()
            print (counter)

        previous_film_url = make_movie_url(movie_href)

    if len(movies_batch) > 0:
        write_batch_movies(movies_batch, filename_movies_extented)
        movies_batch = list()

    print (counter)
    
    movies_ids_hrefs = get_movies_ids_hrefs_to_process(filename_movies) - get_already_processed_movies(filename_movies_extented)
    while len(movies_ids_hrefs) == 0:
        time.sleep(300)
        movies_ids_hrefs = get_movies_ids_hrefs_to_process(filename_movies) - get_already_processed_movies(filename_movies_extented)